In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter


In [6]:
# # DB libraries
# import psycopg2
# from postgres_data import host
# from postgres_data import password
# from postgres_data import user
# from sqlalchemy import create_engine

In [4]:
file_path = Path('../Tables/meta_ml.csv')
df = pd.read_csv(file_path)[:-2]

In [5]:
# ML libraries
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [6]:
# # Create an engine instance
# alchemyEngine = create_engine(f'postgresql://{user}:{password}@{host}:5432/dfph644rcv92i9')

In [9]:
# # Connect to PostgreSQL server
# dbConnection = alchemyEngine.connect()

# # Read data from PostgreSQL database table and load into a DataFrame instance
# df = pd.read_sql("select * from meta", dbConnection)
# dbConnection.close()

In [10]:
ml_table = df[['movie_id','title','release_date','movie_popularity', 
            'vote_average', 'vote_count','budget','revenue', 
            'ratings','studios','genres','actor_popularity_mean',
            'director_popularity_mean']]
ml_table

,movie_id,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,studios,genres,actor_popularity_mean,director_popularity_mean
0,671,Harry Potter and the Philosopher's Stone,2001-11-16,268.472,7.9,21429,125000000,976475550,PG,"[''Warner Bros. Pictures'', '' Heyday Films'',...","[''Adventure'', ''Fantasy'']",2.993965,2.566
1,557,Spider-Man,2002-05-01,480.954,7.2,14421,139000000,821708551,PG-13,"[''Other'', '' Columbia Pictures'', '' Sony Pi...","[''Fantasy'', ''Action'']",2.387022,2.914
2,672,Harry Potter and the Chamber of Secrets,2002-11-13,246.027,7.7,17294,100000000,876688482,PG,"[''Warner Bros. Pictures'', '' Heyday Films'',...","[''Adventure'', ''Fantasy'']",2.696712,2.566
3,673,Harry Potter and the Prisoner of Azkaban,2004-05-31,225.882,8.0,17001,130000000,789804554,PG,"[''Warner Bros. Pictures'', '' Other'', '' Hey...","[''Adventure'', ''Fantasy'']",3.523069,3.333
4,674,Harry Potter and the Goblet of Fire,2005-11-16,244.428,7.8,16341,150000000,895921036,PG-13,"[''Warner Bros. Pictures'', '' Heyday Films'',...","[''Adventure'', ''Fantasy'', ''Family'']",3.234944,2.695
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,370172,No Time to Die,2021-09-29,3366.389,7.6,2075,242000000,734000000,PG-13,"[''Other'', '' Metro-Goldwyn-Mayer'', '' Other...","[''Adventure'', ''Action'', ''Thriller'']",4.164382,4.167
68,580489,Venom: Let There Be Carnage,2021-09-30,5797.863,7.0,2452,110000000,454000000,PG-13,"[''Marvel Entertainment'', '' Pascal Pictures'...","[''Science Fiction'', ''Action'', ''Adventure'']",4.363038,13.077
69,610253,Halloween Kills,2021-10-14,616.978,6.9,1251,20000000,127000000,R,"[''Universal Pictures'', '' Other'', '' Other'...","[''Horror'', ''Thriller'']",3.464000,2.385
70,576845,Last Night in Soho,2021-10-21,685.843,7.5,458,43000000,19000000,R,"[''Other'', '' Other'', '' Other'', '' Other''...","[''Horror'', ''Mystery'', ''Thriller'']",2.869283,7.468


In [11]:
studio_split = []
for studio in ml_table['studios']:
    x = str(studio)[1:-1]
    studio_split.append(x)

studio_array = []
for studio in studio_split:
    x = studio.split(',')
    studio_array.append(x)


ml_table.drop(['studios'],axis=1,inplace=True)
ml_table['studios'] = studio_array

genre_split = []
for genre in ml_table['genres']:
    x = str(genre)[1:-1]
    genre_split.append(x)
genre_split

genre_array=[]
for genre in genre_split:
    x = genre.split(',')
    genre_array.append(x)

ml_table.drop(['genres'],axis=1,inplace=True)
ml_table['genres'] = genre_array

ml_table = ml_table.reindex(columns=['movie_id','title','release_date','movie_popularity', 
            'vote_average', 'vote_count','budget','revenue', 
            'ratings','studios','genres','actor_popularity_mean',
            'director_popularity_mean'])


In [12]:
studio_revised = []
for test in ml_table['studios']:
    holder = []
    for studio in test:
        word = studio.replace('\'','')
        word = studio.replace(' ','')
        if word not in holder:
            holder.append(word)
    studio_revised.append(holder)

genre_revised = []
for test in ml_table['genres']:
    holder = []
    for genre in test:
        word = genre.replace('\'','')
        if (word not in holder and word != ' Other'):
            holder.append(word)
    genre_revised.append(holder)   

ml_table.drop(['studios'],axis=1,inplace=True)
ml_table['studios'] = studio_revised

ml_table.drop(['genres'],axis=1,inplace=True)
ml_table['genres'] = genre_revised

ml_table.head(10)

,movie_id,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_popularity_mean,director_popularity_mean,studios,genres
0,671,Harry Potter and the Philosopher's Stone,2001-11-16,268.472,7.9,21429,125000000,976475550,PG,2.993965,2.566,"['WarnerBros.Pictures', 'HeydayFilms', 'Other']","[Adventure, Fantasy]"
1,557,Spider-Man,2002-05-01,480.954,7.2,14421,139000000,821708551,PG-13,2.387022,2.914,"['Other', 'ColumbiaPictures', 'SonyPictures', ...","[Fantasy, Action]"
2,672,Harry Potter and the Chamber of Secrets,2002-11-13,246.027,7.7,17294,100000000,876688482,PG,2.696712,2.566,"['WarnerBros.Pictures', 'HeydayFilms', 'Other']","[Adventure, Fantasy]"
3,673,Harry Potter and the Prisoner of Azkaban,2004-05-31,225.882,8.0,17001,130000000,789804554,PG,3.523069,3.333,"['WarnerBros.Pictures', 'Other', 'HeydayFilms']","[Adventure, Fantasy]"
4,674,Harry Potter and the Goblet of Fire,2005-11-16,244.428,7.8,16341,150000000,895921036,PG-13,3.234944,2.695,"['WarnerBros.Pictures', 'HeydayFilms', 'Other']","[Adventure, Fantasy, Family]"
5,559,Spider-Man 3,2007-05-01,556.841,6.3,10570,258000000,890871626,PG-13,2.012621,2.914,"[Columbia Pictures, Other, Marvel Studios]","[Fantasy, Action, Adventure]"
6,675,Harry Potter and the Order of the Phoenix,2007-06-28,195.719,7.7,15557,150000000,938212738,PG-13,3.175951,2.813,"[Warner Bros. Pictures, Heyday Films]","[Adventure, Fantasy, Mystery]"
7,8960,Hancock,2008-07-01,216.488,6.3,7702,150000000,629443428,PG-13,2.602644,3.255,"[Other, Columbia Pictures, Other, Sony P...","[Fantasy, Action]"
8,13811,Knowing,2009-03-19,275.317,6.1,4093,50000000,155446362,PG-13,2.835435,0.600,"[Other, Other]","[Action, Mystery, Science Fiction, Thriller]"
9,15092,Crank: High Voltage,2009-04-16,177.734,6.0,2132,20000000,34560577,R,2.050573,1.438,"[Other, Other]","[Action, Thriller, Crime]"


In [13]:
ml_table.to_csv('../Tables/Ml_Test_pgs_checked.csv', index = False)